## BlackCat Organizations
5/19
* Most recent records
* Transit agencies that have fixed-route services (more than the vendors contracted to  provide dial-a-ride / paratransit services—I recall there being a bunch of those in the  BlackCat export I got last year).
* 5311, SGR then + Clovis

In [ ]:
import fuzzywuzzy
import pandas as pd
import siuba  # need this to do type hint in functions
from calitp_data_analysis.sql import to_snakecase
from calitp_data_analysis.tables import tbls
from fuzzywuzzy import process
from siuba import *

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
gcs_path = "gs://calitp-analytics-data/data-analyses/grant_misc/"

### Blackcat

In [ ]:
blackcat = to_snakecase(
    pd.read_excel(f"{gcs_path}BlackCat_Grants_Projects_5_22_23.xlsx")
)

In [ ]:
blackcat.sample()

In [ ]:
blackcat.shape

In [ ]:
# Filter grant fiscal year
blackcat2 = blackcat[blackcat.grant_fiscal_year >= 2018].reset_index(drop=True)

In [ ]:
blackcat2.shape, blackcat2.grant_fiscal_year.value_counts()

In [ ]:
blackcat2.columns

In [ ]:
grants_subset = [
    "5311(f) Cont",
    "CMAQ (FTA 5311)",
    "Section 5311",
    "5311(f) Round 2",
    "5339 (State)",
    "Section 5311(f)",
]

In [ ]:
blackcat2 = blackcat2[blackcat2.funding_program.isin(grants_subset)]

In [ ]:
# Subset to only organizations
organizations = (
    blackcat2[["organization_name", "grant_fiscal_year", "funding_program"]]
    .sort_values(by=["organization_name", "grant_fiscal_year"], ascending=[True, False])
    .drop_duplicates(subset=["organization_name", "funding_program"])
    .reset_index(drop=True)
)

In [ ]:
len(organizations)

In [ ]:
def summarize_rows(df, col_to_group: str, col_to_summarize: str) -> pd.DataFrame:
    """
    Puts all the elements in the column "col to summarize"
    onto one line and separates them by commas.
    """
    df = df.groupby(col_to_group)[col_to_summarize].apply(",".join).reset_index()
    return df

In [ ]:
organizations = summarize_rows(
    organizations, ["organization_name", "grant_fiscal_year"], "funding_program"
)

In [ ]:
organizations = (
    organizations.sort_values(
        by=["organization_name", "grant_fiscal_year"], ascending=[True, False]
    )
    .drop_duplicates(subset=["organization_name"])
    .reset_index(drop=True)
)

In [ ]:
organizations.shape, organizations.organization_name.nunique()

In [ ]:
organizations

### State of Good Repair

In [ ]:
sgr = to_snakecase(pd.read_excel(f"{gcs_path}SGR Calsmart-user-list request.xls"))

In [ ]:
sgr_subset = ["first_name", "last_name", "email", "phone", "title", "agency"]

In [ ]:
sgr2 = sgr[sgr_subset]

In [ ]:
sgr2 = sgr2.drop_duplicates("agency").reset_index(drop=True)

In [ ]:
sgr2.agency.value_counts().head()

In [ ]:
len(sgr2), len(sgr)

In [ ]:
sgr2["funding_program"] = "State of Good Repair"

In [ ]:
sgr2.shape, sgr.agency.nunique()

### Merge BlackCat w/ SGR

In [ ]:
def clean_punctuation(df, agency_col: str) -> pd.DataFrame:
    """
    Cleans up agency names. Assume anything after comma/()/
    ; are acronyms and delete them. Correct certain mispellings.
    Change agency names to title case. Clean whitespaces.
    """
    df[agency_col] = (
        df[agency_col]
        .str.strip()
        .str.split(",")
        .str[0]
        .str.replace("/", "")
        .str.split("(")
        .str[0]
        .str.split("/")
        .str[0]
        .str.split(";")
        .str[0]
        .str.title()
        .str.replace("Trasit", "Transit")
        .str.replace("*", "")
        .str.replace("Agency", "")
        .str.strip()  # strip whitespaces again after getting rid of certain things
    )
    return df

In [ ]:
def flip_county_city(df, agency_col: str):
    # https://github.com/cal-itp/data-analyses/blob/main/Agreement_Overlap/add_dla.ipynb
    to_correct = df[
        (df[agency_col].str.contains("County")) | (df[agency_col].str.contains("City"))
    ]
    to_correct = to_correct[[agency_col]].drop_duplicates().reset_index(drop=True)
    to_correct["str_len"] = to_correct[agency_col].str.split().str.len()
    to_correct = to_correct[to_correct.str_len <= 5].reset_index(drop=True)
    to_correct[["name_pt1", "name_pt2"]] = to_correct[agency_col].str.split(
        " Of ", 1, expand=True
    )
    to_correct["new_name"] = to_correct["name_pt2"] + " " + to_correct["name_pt1"]

    new_names_dictionary = dict(to_correct[[agency_col, "new_name"]].values)
    df["agency_corrected"] = df[agency_col].map(new_names_dictionary)
    df["agency_corrected"] = df["agency_corrected"].fillna(df[agency_col])

    df = df.drop(columns=[agency_col])
    df = df.rename(columns={"agency_corrected": agency_col})

    return df

In [ ]:
def clean_organization_names(df, agency_col: str):
    df = clean_punctuation(df, agency_col)
    df = flip_county_city(df, agency_col)
    return df

In [ ]:
organizations = clean_organization_names(organizations, "organization_name")

In [ ]:
len(organizations)

In [ ]:
sgr2 = clean_organization_names(sgr2, "agency")

In [ ]:
def replace_matches_set_ratio(df, column, new_col_name, string_to_match, min_ratio):
    # Get a list of unique strings
    strings = df[column].unique()

    # Get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(
        string_to_match, strings, limit=10, scorer=fuzzywuzzy.fuzz.token_set_ratio
    )

    # Only get matches with a  min ratio
    close_matches = [matches[0] for matches in matches if matches[1] > min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches
    df.loc[rows_with_matches, new_col_name] = string_to_match

In [ ]:
def find_fuzzy_match(
    df1,
    df2,
    df1_fuzzy_column: str,
    df2_fuzzy_column: str,
    new_column: str,
    min_ratio: int,
):
    unique_values = df1[df1_fuzzy_column].unique().tolist()
    for i in unique_values:
        replace_matches_set_ratio(df2, df2_fuzzy_column, new_column, i, min_ratio)
    return df2

In [ ]:
organizations = find_fuzzy_match(
    sgr, organizations, "agency", "organization_name", "fuzzy_match_agency", 95
)

In [ ]:
organizations

In [ ]:
fuzzy_to_del = [
    "Amador Transit",
    "Eastern Contra Costa Transit Authority",
    "Madera County",
]

In [ ]:
for i in fuzzy_to_del:
    organizations.loc[
        organizations["organization_name"].eq(i), "fuzzy_match_agency"
    ] = None

In [ ]:
organizations.fuzzy_match_agency = organizations.fuzzy_match_agency.fillna(
    organizations.organization_name
)

In [ ]:
# organizations.sort_values(by = ['organization_name'])

In [ ]:
# organizations = organizations.drop(columns = ['organization_name']).rename(columns = {'fuzzy_match_agency':'organization_name'})

In [ ]:
m1 = pd.merge(
    organizations,
    sgr2,
    left_on=["fuzzy_match_agency"],
    right_on=["agency"],
    how="outer",
    indicator=True,
)

In [ ]:
m1._merge.value_counts()

In [ ]:
m1.funding_program_x = m1.funding_program_x.fillna("State of Good Repair")

In [ ]:
m1.funding_program_y = m1.funding_program_y.fillna(m1.funding_program_x)

In [ ]:
m1["funding_program"] = m1.funding_program_x + "," + m1.funding_program_y

In [ ]:
m1.shape, m1.organization_name.nunique()

In [ ]:
m1.organization_name = m1.organization_name.fillna(m1.agency)

In [ ]:
m1.sort_values(by = ['organization_name', 'agency'])

In [ ]:
m1.funding_program = m1.funding_program.fillna(m1.funding_program_y)

In [ ]:
cols_to_drop = [
    "funding_program_x",
    "funding_program_y",
    "fuzzy_match_agency",
    "agency",
    "grant_fiscal_year",
]

In [ ]:
m1 = m1.drop(columns=cols_to_drop)

In [ ]:
m1.organization_name.nunique(), m1.shape

In [ ]:
m2 = summarize_rows(
    m1,
    ["organization_name"],
    "funding_program",
)

In [ ]:
m2 = pd.merge(m2, m1.drop(columns = ['funding_program']), on = ['organization_name'], how = "left")

In [ ]:
m2 = m2.sort_values(['organization_name']).drop_duplicates(subset = ['organization_name'])

In [ ]:
m2.organization_name.nunique(), m2.shape

### Airtable
* Grab only fixed route providers.

In [ ]:
airtable = tbls.external_airtable.california_transit__services() >> collect()

In [ ]:
# airtable.columns

In [ ]:
# ['name','service_type', 'service_operator_type', 'operator_organization_type']
airtable_subset = ["name", "service_type"]

In [ ]:
airtable2 = airtable[airtable_subset]

In [ ]:
airtable2.name.nunique()

In [ ]:
airtable2.info()

In [ ]:
airtable2.sort_values(["name"]).head()

In [ ]:
airtable3 = airtable2.explode("service_type").reset_index(drop=True)

In [ ]:
airtable3.head()

In [ ]:
airtable3.service_type = airtable3.service_type.fillna("NA")

In [ ]:
# fixed route only
#airtable4 = airtable3[airtable3.service_type.str.lower().str.contains("fixed")]

In [ ]:
airtable5 = (
    airtable3.drop_duplicates(["name"]).sort_values(by=["name"]).reset_index(drop=True)
)

In [ ]:
len(airtable5)

In [ ]:
airtable5.name.nunique()

In [ ]:
airtable5 = airtable5.fillna('NA')

In [ ]:
airtable5.info()

### Merge

In [ ]:
airtable5 = clean_organization_names(airtable5, "name")

In [ ]:
#airtable5.sort_values('name').head(20)

In [ ]:
m2 = find_fuzzy_match(airtable5, m2, "name", "organization_name", "fuzzy_agency", 95)

In [ ]:
for i in [
    "Eastern Contra Costa Transit Authority",
    "Fresno County",
    "Livermore Amador Valley Transit Authority",
    "Eastern Sierra Transit Authority",
    "Alpine County Local Transportation Commission"
]:
    m2.loc[m2["organization_name"].eq(i), "fuzzy_agency"] = None

In [ ]:
m2.loc[m2.fuzzy_agency.notna()][["organization_name", "fuzzy_agency"]].sort_values(
    ["organization_name"]
)

In [ ]:
# Reverse -> replace
"""
for i in airtable5.name.unique().tolist():
       replace_matches_set_ratio(
        organizations, "organization_name", "project_name_fuzzy", i, 95) """

In [ ]:
# Fuzzy matches that didn't work
# Use organization name
fuzzy_matches_to_filter = [
    "Eastern Contra Costa Transit Authority",
    "Livermore Amador Valley Transit Authority",
    "Calaveras Transit",
    "City Of Corcoran - Corcoran Area Transit",
]

In [ ]:
len(m2)

In [ ]:
# These are the fuzzy matches that worked.
found_matches = (
    m2[((~m2.fuzzy_agency.isna()) & (~m2.fuzzy_agency.isin(fuzzy_matches_to_filter)))]
).reset_index(drop=True)

In [ ]:
len(found_matches)

In [ ]:
still_need_matches = m2[(~m2.organization_name.isin(found_matches.organization_name.tolist()))]

In [ ]:
found_matches = found_matches.drop(columns = ['organization_name']).rename(columns = {'fuzzy_agency':'organization_name'})

In [ ]:
found_matches

* Calaveras Transit change to Calaveras Connect
* Arvin City  Arvin Transit
* Auburn City Auburn Transit
* County Of Los Angeles - Department Of Public Works, Los Angeles County Transit Services
* 34	County Of Sacramento Department Of Transportation Sacrt Bus
* County Of Shasta Department Of Public Works
* Dinuba City Dinuba Connection
* Lassen Transit Service Lassen Rural Bus
* Needles City  Needles Area Transit
* Nevada Public Works County Nevada County Connects
* Ojai City Ojai Trolley
* Palo Verde Valley Transit Palos Verdes Peninsula Transit Authority
* Placer County Public Works Placer County Transit
* Plumas County Transportation Commission Plumas Transit Systems
* Porterville City  Porterville Transit
* Ridgecrest City Ridgecrest Transit
* Rio Vista City Rio Vista Delta Breeze
* Santa Maria City Santa Maria Regional Transit
* Siskiyou County Siskiyou Transit And General Express
* Stanislaus County Public Works - Transit Division  Stanislaus Regional Transit Authority
* Taft City Taft Area Transit
* Tehama County Transit  Tehama Rural Area Express
* Transportation Trinity County Department Trinity Transit
* Transit Joint Powers Authority For Merced County Merced The Bus
* Visalia City Visilia Transit
* Yolo County Transportation District Yolobus


In [ ]:
to_map = {
    "Tulare County": "Tuolumne County Transit",
    "Turlock City": "Turlock Transit",
    "Union City City": "Union City Transit",
    "Calaveras Transit": "Calaveras Connect",
    "Alameda-Contra Costa Transit District": "Ac Transit",
    "Arcadia City": "Arcadia Transit",
    "Banning City": "Banning Pass Transit",
    "Beaumont City": "Beaumont Pass Transit",
    "Calaveras Council Of Governments": "Calaveras Connect",
    "Camarillo City": "Camarillo Area Transit",
    "Commerce City": "Commerce Municipal Bus Lines",
    "Corona City": "Corona Cruiser",
    "Delano City": "Delano Area Rapid Transit",
    "Eastern Sierra Transit Authority": "Eastern Sierra Transit Authority Community Routes",
    "Elk Grove City": "Elk Grove Transit Services",
    "Fairfield City": "Fairfield And Suisun Transit",
    "Folsom City": "Folsom Stage Line",
    "Glenn County": "Glenn Ride",
    "Guadalupe City": "Guadalupe Flyer",
    "Lassen County": "Lassen Rural Bus",
    "Marin County Transit District": "Marin Transit",
    "Madera County": "Madera Metro",
    "Mariposa County": "Mariposa Grove Shuttle",
    "Morro Bay City": "Morro Bay Transit",
    "Norwalk City": "Norwalk Transit System",
    "Roseville City": "Roseville Transit",
    "Sacramento Regional Transit District": "Sacramento Regional Transit District Bus",
    "San Diego City": "San Diego Trolley",
    "San Francisco City": "Muni Bus",
    "Santa Rosa City": "Santa Rosa Citybus",
    "Shafter City": "Shafter Dial-A-Ride",
    "Sierra County": "Sierra Point Shuttle",
    "Simi Valley City": "Simi Valley Transit",
    "Sonoma Marin Area Rail Transit": "Sonoma-Marin Area Rail Transit",
    "Arvin City": "Arvin Transit",
    "Auburn City": "Auburn Transit",
    "County Of Los Angeles - Department Of Public Works": "Los Angeles County Transit Services",
    "County Of Sacramento Department Of Transportation": "Sacrt Bus",
    "Dinuba City": "Dinuba Connection",
    "Lassen Transit Service": "Lassen Rural Bus",
    "Needles City": "Needles Area Transit",
    "Nevada Public Works": "County Nevada County Connects",
    "Ojai City": "Ojai Trolley",
    "Palo Verde Valley Transit": "Palos Verdes Peninsula Transit Authority",
    "Placer County Public Works": "Placer County Transit",
    "Plumas County Transportation Commission": "Plumas Transit Systems",
    "Porterville City": "Porterville Transit",
    "Ridgecrest City": "Ridgecrest Transit",
    "Rio Vista City": "Rio Vista Delta Breeze",
    "Santa Maria City": "Santa Maria Regional Transit",
    "Siskiyou County": "Siskiyou Transit And General Express",
    "Taft City": "Taft Area Transit",
    "Tehama County Transit": "Tehama Rural Area Express",
    "Transportation Trinity County Department": "Trinity Transit",
    "Transit Joint Powers Authority For Merced": "County Merced The Bus",
    "Visalia City": "Visilia Transit",
    "Yolo County Transportation District": "Yolobus",
}

In [ ]:
still_need_matches.organization_name = still_need_matches.organization_name.replace(
    to_map
)

In [ ]:
still_need_matches.head()

In [ ]:
# Concat
blackcat_cleaned = pd.concat([found_matches, still_need_matches], axis=0)

In [ ]:
len(blackcat_cleaned)

In [ ]:
blackcat_cleaned.organization_name = blackcat_cleaned.organization_name.replace(
    to_map
)

In [ ]:
m3 = pd.merge(
    blackcat_cleaned,
    airtable5,
    how="left",
    left_on="organization_name",
    right_on="name",
)

In [ ]:
m3.service_type = m3.service_type.fillna("no service info")

In [ ]:
m3 = m3.drop(columns=["name"])

In [ ]:
m3.service_type.value_counts()

In [ ]:
m3 = m3.fillna("NA")

In [ ]:
final_subset = [
    "funding_program",
    "organization_name",
    "first_name",
    "last_name",
    "email",
    "phone",
    "title",
    "service_type",
]

In [ ]:
m3 = m3[final_subset]

In [ ]:
def delete_repeated_element(df, col: str):
    df[col] = (
        df[col]
        .apply(lambda x: ", ".join(set([y.strip() for y in x.split(",")])))
        .str.strip()
    )
    return df

In [ ]:
m3.service_type = m3.service_type.str.title()

In [ ]:
m3 = delete_repeated_element(m3, "funding_program")

In [ ]:
m3 = m3.drop_duplicates(subset = ['organization_name']).sort_values(["organization_name"]).reset_index(drop=True)

In [ ]:
m3.shape, m3.organization_name.nunique()

In [ ]:
m3.head()

In [ ]:
m3.to_excel(f"{gcs_path}5311_SGR_Recipients.xlsx")

In [ ]:
m3.organization_name.tolist()